In [382]:
import pandas as pd
import numpy as np
import ast
from bs4 import BeautifulSoup
import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
ps = PorterStemmer()
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [2]:
#importing the dataset

# dataset_path = 'C:/Users/ha468/OneDrive/Desktop/Work/My Datasets/dataset_withCategory_test.csv'
dataset_path = 'C:/Users/ha468/OneDrive/Desktop/Work/My Datasets/dataset_withCategory.csv'


df = pd.read_csv(dataset_path)

In [65]:
df

,product_name,product_price,product_image,product_link,product_store,product_category,product_ratings,rating_count,description,date,rating_weighted,product_id,tags
0,Mibro A2 Bluetooth Calling Smart Watch,8299.0,https://images.priceoye.pk/mibro-a2-bluetooth-...,https://priceoye.pk/smart-watches/mibro/mibro-...,PriceOye,"Smart-Watches, Mibro",5.000000,8.00000,"{'Display': '1.39 inches', 'Screen Type': 'HD ...",2023-12-03,40.000000,1,mibro a2 bluetooth calling smart watch display...
1,Mibro Watch Lite2,12999.0,https://images.priceoye.pk/mibro-watch-lite2-p...,https://priceoye.pk/smart-watches/mibro/mibro-...,PriceOye,"Smart-Watches, Mibro",5.000000,79.00000,"{'Display': '1.3 Inches', 'Screen Type': 'Amol...",2023-12-03,395.000000,2,mibro watch lite2 display 1 3 inch screen type...
2,Mibro GS Pro Calling Smart Watch,15999.0,https://images.priceoye.pk/mibro-gs-pro-callin...,https://priceoye.pk/smart-watches/mibro/mibro-...,PriceOye,"Smart-Watches, Mibro",5.000000,5.00000,"{'Display': '1.43', 'Screen Type': 'AMOLED', '...",2023-12-03,25.000000,3,mibro g pro calling smart watch display 1 43 s...
3,Mibro T1 Smart Watch,13999.0,https://images.priceoye.pk/mibro-t1-pakistan-p...,https://priceoye.pk/smart-watches/mibro/mibro-t1,PriceOye,"Smart-Watches, Mibro",5.000000,79.00000,"{'Display': '1.6 Inch', 'Screen Type': 'AMOLED...",2023-12-03,395.000000,4,mibro t1 smart watch display 1 6 inch screen t...
4,Mibro Smart Watch T2,13999.0,https://images.priceoye.pk/mibro-smart-watch-t...,https://priceoye.pk/smart-watches/mibro/mibro-...,PriceOye,"Smart-Watches, Mibro",5.000000,1.00000,"{'Display': '1.75 Inches', 'Screen Type': 'AMO...",2023-12-03,5.000000,5,mibro smart watch t2 display 1 75 inch screen ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3686,Oppo A77s (4G 8GB 128GB Starry Black) With Off...,74799.0,https://cdn.homeshopping.pk/product_images/d/8...,https://homeshopping.pk/products/Oppo-A77s-8GB...,HomeShopping,"MobilePhones, Oppo",4.631924,17.65277,"['', 'Oppo A77s\xa0Price & Specs', 'HIGHLIGHT'...",2023-12-03,81.766283,3687,oppo a77s 4g 8gb 128gb starry black with offic...
3687,OPPO A16k (4G 4GB 64GB Blue) With Official War...,38999.0,https://cdn.homeshopping.pk/product_images/t/8...,https://homeshopping.pk/products/OPPO-A16k-Blu...,HomeShopping,"MobilePhones, Oppo",4.631924,17.65277,"['', 'OPPO A16k Price & Specs', 'OPPO A16k Pri...",2023-12-03,81.766283,3688,oppo a16k 4g 4gb 64gb blue with official warra...
3688,Nokia 105 simba (2022) wireless FM radio dual sim,6299.0,https://cdn.homeshopping.pk/product_images/i/9...,https://homeshopping.pk/products/Nokia-105-sim...,HomeShopping,"MobilePhones, Nokia",4.631924,17.65277,"['', 'Nokia 105 simba (2022) wireless FM radio...",2023-12-03,81.766283,3689,nokia 105 simba 2022 wireless fm radio dual si...
3689,Nokia 5710 (4G Dual Sim 2.4 inch Black) With O...,18599.0,https://cdn.homeshopping.pk/product_images/u/7...,https://homeshopping.pk/products/Nokia-5710-Bl...,HomeShopping,"MobilePhones, Nokia",4.631924,17.65277,"['', 'Xiaomi Redmi Note 12\xa0Price & Specs', ...",2023-12-03,81.766283,3690,nokia 5710 4g dual sim 2 4 inch black with off...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4278 entries, 0 to 4277
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   product_name      4278 non-null   object 
 1   product_price     4278 non-null   object 
 2   product_image     4278 non-null   object 
 3   product_link      4278 non-null   object 
 4   product_store     4278 non-null   object 
 5   product_category  4278 non-null   object 
 6   product_ratings   1679 non-null   float64
 7   rating_count      1679 non-null   object 
 8   description       4278 non-null   object 
 9   date              4278 non-null   object 
dtypes: float64(1), object(9)
memory usage: 334.3+ KB


#  ** DATA CLEANING **

## 1. REMOVING NULL VALUES

In [5]:
df.isnull().sum()

product_name           0
product_price          0
product_image          0
product_link           0
product_store          0
product_category       0
product_ratings     2599
rating_count        2599
description            0
date                   0
dtype: int64

In [6]:
# Convert the rating count to Float to calculate the mean of ratings and ratings count because it can be more informative 
# then using zeros as replacement
df['rating_count'] = df['rating_count'].str.extract('(\d+)').astype(float)

In [7]:
mean_ratings = df['product_ratings'].mean()
mean_rating_count = df['rating_count'].mean()

df['product_ratings'].fillna(mean_ratings, inplace=True)
df['rating_count'].fillna(mean_rating_count, inplace=True)

In [8]:
df.isnull().sum()

product_name        0
product_price       0
product_image       0
product_link        0
product_store       0
product_category    0
product_ratings     0
rating_count        0
description         0
date                0
dtype: int64

In [9]:
df

,product_name,product_price,product_image,product_link,product_store,product_category,product_ratings,rating_count,description,date
0,Mibro A2 Bluetooth Calling Smart Watch,"Rs. 8,299",https://images.priceoye.pk/mibro-a2-bluetooth-...,https://priceoye.pk/smart-watches/mibro/mibro-...,PriceOye,"Smart-Watches, Mibro",5.000000,8.00000,"{'Display': '1.39 inches', 'Screen Type': 'HD ...",2023-12-03
1,Mibro Watch Lite2,"Rs. 12,999",https://images.priceoye.pk/mibro-watch-lite2-p...,https://priceoye.pk/smart-watches/mibro/mibro-...,PriceOye,"Smart-Watches, Mibro",5.000000,79.00000,"{'Display': '1.3 Inches', 'Screen Type': 'Amol...",2023-12-03
2,Mibro GS Pro Calling Smart Watch,"Rs. 15,999",https://images.priceoye.pk/mibro-gs-pro-callin...,https://priceoye.pk/smart-watches/mibro/mibro-...,PriceOye,"Smart-Watches, Mibro",5.000000,5.00000,"{'Display': '1.43', 'Screen Type': 'AMOLED', '...",2023-12-03
3,Mibro T1 Smart Watch,"Rs. 13,999",https://images.priceoye.pk/mibro-t1-pakistan-p...,https://priceoye.pk/smart-watches/mibro/mibro-t1,PriceOye,"Smart-Watches, Mibro",5.000000,79.00000,"{'Display': '1.6 Inch', 'Screen Type': 'AMOLED...",2023-12-03
4,Mibro Smart Watch T2,"Rs. 13,999",https://images.priceoye.pk/mibro-smart-watch-t...,https://priceoye.pk/smart-watches/mibro/mibro-...,PriceOye,"Smart-Watches, Mibro",5.000000,1.00000,"{'Display': '1.75 Inches', 'Screen Type': 'AMO...",2023-12-03
...,...,...,...,...,...,...,...,...,...,...
4273,Oppo A77s (4G 8GB 128GB Starry Black) With Off...,"<em class=""ProductPrice VariationProductPrice""...",https://cdn.homeshopping.pk/product_images/d/8...,https://homeshopping.pk/products/Oppo-A77s-8GB...,HomeShopping,"MobilePhones, Oppo",4.631924,17.65277,"['', 'Oppo A77s\xa0Price & Specs', 'HIGHLIGHT'...",2023-12-03
4274,OPPO A16k (4G 4GB 64GB Blue) With Official War...,"<em class=""ProductPrice VariationProductPrice""...",https://cdn.homeshopping.pk/product_images/t/8...,https://homeshopping.pk/products/OPPO-A16k-Blu...,HomeShopping,"MobilePhones, Oppo",4.631924,17.65277,"['', 'OPPO A16k Price & Specs', 'OPPO A16k Pri...",2023-12-03
4275,Nokia 105 simba (2022) wireless FM radio dual sim,"<em class=""ProductPrice VariationProductPrice""...",https://cdn.homeshopping.pk/product_images/i/9...,https://homeshopping.pk/products/Nokia-105-sim...,HomeShopping,"MobilePhones, Nokia",4.631924,17.65277,"['', 'Nokia 105 simba (2022) wireless FM radio...",2023-12-03
4276,Nokia 5710 (4G Dual Sim 2.4 inch Black) With O...,"<em class=""ProductPrice VariationProductPrice""...",https://cdn.homeshopping.pk/product_images/u/7...,https://homeshopping.pk/products/Nokia-5710-Bl...,HomeShopping,"MobilePhones, Nokia",4.631924,17.65277,"['', 'Xiaomi Redmi Note 12\xa0Price & Specs', ...",2023-12-03


## 2. REMOVING DUPLICATES 

In [10]:
df.duplicated().sum()

587

In [11]:
# Remove duplicate rows from the DataFrame
df.drop_duplicates(inplace=True)

In [12]:
df.duplicated().sum()

0

In [13]:
df

,product_name,product_price,product_image,product_link,product_store,product_category,product_ratings,rating_count,description,date
0,Mibro A2 Bluetooth Calling Smart Watch,"Rs. 8,299",https://images.priceoye.pk/mibro-a2-bluetooth-...,https://priceoye.pk/smart-watches/mibro/mibro-...,PriceOye,"Smart-Watches, Mibro",5.000000,8.00000,"{'Display': '1.39 inches', 'Screen Type': 'HD ...",2023-12-03
1,Mibro Watch Lite2,"Rs. 12,999",https://images.priceoye.pk/mibro-watch-lite2-p...,https://priceoye.pk/smart-watches/mibro/mibro-...,PriceOye,"Smart-Watches, Mibro",5.000000,79.00000,"{'Display': '1.3 Inches', 'Screen Type': 'Amol...",2023-12-03
2,Mibro GS Pro Calling Smart Watch,"Rs. 15,999",https://images.priceoye.pk/mibro-gs-pro-callin...,https://priceoye.pk/smart-watches/mibro/mibro-...,PriceOye,"Smart-Watches, Mibro",5.000000,5.00000,"{'Display': '1.43', 'Screen Type': 'AMOLED', '...",2023-12-03
3,Mibro T1 Smart Watch,"Rs. 13,999",https://images.priceoye.pk/mibro-t1-pakistan-p...,https://priceoye.pk/smart-watches/mibro/mibro-t1,PriceOye,"Smart-Watches, Mibro",5.000000,79.00000,"{'Display': '1.6 Inch', 'Screen Type': 'AMOLED...",2023-12-03
4,Mibro Smart Watch T2,"Rs. 13,999",https://images.priceoye.pk/mibro-smart-watch-t...,https://priceoye.pk/smart-watches/mibro/mibro-...,PriceOye,"Smart-Watches, Mibro",5.000000,1.00000,"{'Display': '1.75 Inches', 'Screen Type': 'AMO...",2023-12-03
...,...,...,...,...,...,...,...,...,...,...
4273,Oppo A77s (4G 8GB 128GB Starry Black) With Off...,"<em class=""ProductPrice VariationProductPrice""...",https://cdn.homeshopping.pk/product_images/d/8...,https://homeshopping.pk/products/Oppo-A77s-8GB...,HomeShopping,"MobilePhones, Oppo",4.631924,17.65277,"['', 'Oppo A77s\xa0Price & Specs', 'HIGHLIGHT'...",2023-12-03
4274,OPPO A16k (4G 4GB 64GB Blue) With Official War...,"<em class=""ProductPrice VariationProductPrice""...",https://cdn.homeshopping.pk/product_images/t/8...,https://homeshopping.pk/products/OPPO-A16k-Blu...,HomeShopping,"MobilePhones, Oppo",4.631924,17.65277,"['', 'OPPO A16k Price & Specs', 'OPPO A16k Pri...",2023-12-03
4275,Nokia 105 simba (2022) wireless FM radio dual sim,"<em class=""ProductPrice VariationProductPrice""...",https://cdn.homeshopping.pk/product_images/i/9...,https://homeshopping.pk/products/Nokia-105-sim...,HomeShopping,"MobilePhones, Nokia",4.631924,17.65277,"['', 'Nokia 105 simba (2022) wireless FM radio...",2023-12-03
4276,Nokia 5710 (4G Dual Sim 2.4 inch Black) With O...,"<em class=""ProductPrice VariationProductPrice""...",https://cdn.homeshopping.pk/product_images/u/7...,https://homeshopping.pk/products/Nokia-5710-Bl...,HomeShopping,"MobilePhones, Nokia",4.631924,17.65277,"['', 'Xiaomi Redmi Note 12\xa0Price & Specs', ...",2023-12-03


## 3. MAKING DATASET CONSISTENT

In [14]:
df['description']

0       {'Display': '1.39 inches', 'Screen Type': 'HD ...
1       {'Display': '1.3 Inches', 'Screen Type': 'Amol...
2       {'Display': '1.43', 'Screen Type': 'AMOLED', '...
3       {'Display': '1.6 Inch', 'Screen Type': 'AMOLED...
4       {'Display': '1.75 Inches', 'Screen Type': 'AMO...
                              ...                        
4273    ['', 'Oppo A77s\xa0Price & Specs', 'HIGHLIGHT'...
4274    ['', 'OPPO A16k Price & Specs', 'OPPO A16k Pri...
4275    ['', 'Nokia 105 simba (2022) wireless FM radio...
4276    ['', 'Xiaomi Redmi Note 12\xa0Price & Specs', ...
4277    ['', 'Google Pixel 8 Pro\xa0Price & Specs', 'G...
Name: description, Length: 3691, dtype: object

In [15]:
def convert_to_string(entry):
    try:
        # Try to evaluate as a dictionary
        entry_dict = ast.literal_eval(entry)
        if isinstance(entry_dict, (dict, list)):
            return str(entry_dict)
    except (ValueError, SyntaxError):
        # If literal_eval fails, handle non-dictionary or non-list entries
        return str(entry)
    return str(entry)

In [16]:
# Apply the conversion function to the 'description' column
df['description'] = df['description'].apply(convert_to_string)

In [17]:
df['description']

0       {'Display': '1.39 inches', 'Screen Type': 'HD ...
1       {'Display': '1.3 Inches', 'Screen Type': 'Amol...
2       {'Display': '1.43', 'Screen Type': 'AMOLED', '...
3       {'Display': '1.6 Inch', 'Screen Type': 'AMOLED...
4       {'Display': '1.75 Inches', 'Screen Type': 'AMO...
                              ...                        
4273    ['', 'Oppo A77s\xa0Price & Specs', 'HIGHLIGHT'...
4274    ['', 'OPPO A16k Price & Specs', 'OPPO A16k Pri...
4275    ['', 'Nokia 105 simba (2022) wireless FM radio...
4276    ['', 'Xiaomi Redmi Note 12\xa0Price & Specs', ...
4277    ['', 'Google Pixel 8 Pro\xa0Price & Specs', 'G...
Name: description, Length: 3691, dtype: object

In [18]:
df['product_price'][4000]

'<em class="ProductPrice VariationProductPrice" itemprop="price"><div class="ActualPrice"> Rs 384,999</div></em>'

In [19]:
# Function to remove any other thing than the price
def extract_price(html):
    if '<div class="ActualPrice">' in html:
        soup = BeautifulSoup(html, 'html.parser')
        actual_price_div = soup.find('div', {'class': 'ActualPrice'})
        return actual_price_div.text.strip() if actual_price_div else None
    else:
        return html

In [20]:
# Apply the function to the 'Product' column
df['product_price'] = df['product_price'].apply(extract_price)

In [21]:
df['product_price'][2000]

'Rs 531,399.00'

In [22]:
# Function to remove the alphabets from the price column 

def clean_and_convert_price(price):
    # Split the string on spaces and take the last part (which should be the numeric part)
    numeric_part = price.split()[-1]
    # Remove currency symbols, commas, and other non-numeric characters
    cleaned_price = re.sub(r'[^\d.]', '', numeric_part)
    return cleaned_price

In [23]:
# Apply the function to the 'price' column
df['product_price'] = df['product_price'].apply(clean_and_convert_price)

In [24]:
df['product_price'] = pd.to_numeric(df['product_price'])

In [25]:
df['product_price'].fillna(0, inplace=True)

In [26]:
df['product_price'].isnull().sum()

0

In [27]:
# Making a rating_weighted column soo that we can sort the product on the basis on weight of rating but also the number of 
# ratings on each product.

df['rating_weighted'] = df['product_ratings'] * df['rating_count']

In [28]:
# Reset the index to create a new 'product_id' column
df.reset_index(drop=True, inplace=True)

# Add 1 to the index to start IDs from 1
df['product_id'] = df.index + 1

In [29]:
df

,product_name,product_price,product_image,product_link,product_store,product_category,product_ratings,rating_count,description,date,rating_weighted,product_id
0,Mibro A2 Bluetooth Calling Smart Watch,8299.0,https://images.priceoye.pk/mibro-a2-bluetooth-...,https://priceoye.pk/smart-watches/mibro/mibro-...,PriceOye,"Smart-Watches, Mibro",5.000000,8.00000,"{'Display': '1.39 inches', 'Screen Type': 'HD ...",2023-12-03,40.000000,1
1,Mibro Watch Lite2,12999.0,https://images.priceoye.pk/mibro-watch-lite2-p...,https://priceoye.pk/smart-watches/mibro/mibro-...,PriceOye,"Smart-Watches, Mibro",5.000000,79.00000,"{'Display': '1.3 Inches', 'Screen Type': 'Amol...",2023-12-03,395.000000,2
2,Mibro GS Pro Calling Smart Watch,15999.0,https://images.priceoye.pk/mibro-gs-pro-callin...,https://priceoye.pk/smart-watches/mibro/mibro-...,PriceOye,"Smart-Watches, Mibro",5.000000,5.00000,"{'Display': '1.43', 'Screen Type': 'AMOLED', '...",2023-12-03,25.000000,3
3,Mibro T1 Smart Watch,13999.0,https://images.priceoye.pk/mibro-t1-pakistan-p...,https://priceoye.pk/smart-watches/mibro/mibro-t1,PriceOye,"Smart-Watches, Mibro",5.000000,79.00000,"{'Display': '1.6 Inch', 'Screen Type': 'AMOLED...",2023-12-03,395.000000,4
4,Mibro Smart Watch T2,13999.0,https://images.priceoye.pk/mibro-smart-watch-t...,https://priceoye.pk/smart-watches/mibro/mibro-...,PriceOye,"Smart-Watches, Mibro",5.000000,1.00000,"{'Display': '1.75 Inches', 'Screen Type': 'AMO...",2023-12-03,5.000000,5
...,...,...,...,...,...,...,...,...,...,...,...,...
3686,Oppo A77s (4G 8GB 128GB Starry Black) With Off...,74799.0,https://cdn.homeshopping.pk/product_images/d/8...,https://homeshopping.pk/products/Oppo-A77s-8GB...,HomeShopping,"MobilePhones, Oppo",4.631924,17.65277,"['', 'Oppo A77s\xa0Price & Specs', 'HIGHLIGHT'...",2023-12-03,81.766283,3687
3687,OPPO A16k (4G 4GB 64GB Blue) With Official War...,38999.0,https://cdn.homeshopping.pk/product_images/t/8...,https://homeshopping.pk/products/OPPO-A16k-Blu...,HomeShopping,"MobilePhones, Oppo",4.631924,17.65277,"['', 'OPPO A16k Price & Specs', 'OPPO A16k Pri...",2023-12-03,81.766283,3688
3688,Nokia 105 simba (2022) wireless FM radio dual sim,6299.0,https://cdn.homeshopping.pk/product_images/i/9...,https://homeshopping.pk/products/Nokia-105-sim...,HomeShopping,"MobilePhones, Nokia",4.631924,17.65277,"['', 'Nokia 105 simba (2022) wireless FM radio...",2023-12-03,81.766283,3689
3689,Nokia 5710 (4G Dual Sim 2.4 inch Black) With O...,18599.0,https://cdn.homeshopping.pk/product_images/u/7...,https://homeshopping.pk/products/Nokia-5710-Bl...,HomeShopping,"MobilePhones, Nokia",4.631924,17.65277,"['', 'Xiaomi Redmi Note 12\xa0Price & Specs', ...",2023-12-03,81.766283,3690


# CONTENT BASED RECOMMENDATIONS

In [30]:
# Making a new DF to not ruin my original cleaned dataset.
newDF = df

In [31]:
# Now i want to make a tags column to make tags for my product by combining the category, name, description, price

newDF['tags'] = newDF['product_name'] + ' ' + newDF['description'] + ' ' + newDF['product_category']


In [32]:
# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Function for lemmatization
def lemmatize_tags(tags):
    tokens = word_tokenize(tags)
    lemmatized_tags = ' '.join([lemmatizer.lemmatize(token) for token in tokens])
    return lemmatized_tags

In [33]:
# Apply lemmatization to the 'tags' column
newDF['tags'] = newDF['tags'].apply(lambda x: lemmatize_tags(re.sub(r'\W', ' ', x.lower())))

In [34]:
newDF['tags']

0       mibro a2 bluetooth calling smart watch display...
1       mibro watch lite2 display 1 3 inch screen type...
2       mibro g pro calling smart watch display 1 43 s...
3       mibro t1 smart watch display 1 6 inch screen t...
4       mibro smart watch t2 display 1 75 inch screen ...
                              ...                        
3686    oppo a77s 4g 8gb 128gb starry black with offic...
3687    oppo a16k 4g 4gb 64gb blue with official warra...
3688    nokia 105 simba 2022 wireless fm radio dual si...
3689    nokia 5710 4g dual sim 2 4 inch black with off...
3690    google pixel 8 pro 5g 12gb 128gb bay non pta g...
Name: tags, Length: 3691, dtype: object

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

## we will be converting our tags into vectors. for that we need to use countvectorizer from sklearn
## we are making cv and we are taking max features to 5000 meaning the amount of words to form a vector and 
## stop_words='english' means we are taking stop words from english lang or dictionary

In [36]:
vectors = cv.fit_transform(newDF['tags']).toarray() ## we are creating a variable vector and stroing our array 

In [37]:
# The similarity matrix contains all similarity score of each item to another
similarity = cosine_similarity(vectors)

In [38]:
def recommendProducts(product_name):
    matching_products = newDF[newDF['product_name'].str.contains(product_name, case=False)]

    recommended_products = []

    if not matching_products.empty:
        product_index = matching_products.index[0]
        distances = similarity[product_index]
        products_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:10]

        for i in products_list:
            product_info = {
                'product_id': newDF.iloc[i[0]].product_id,
                'product_name': newDF.iloc[i[0]].product_name,
                'product_link': newDF.iloc[i[0]].product_link,
                'product_image': newDF.iloc[i[0]].product_image,
                'product_price':newDF.iloc[i[0]].product_price,
                'product_category':newDF.iloc[i[0]].product_category,
                'product_ratings':newDF.iloc[i[0]].product_ratings,
                'product_rating_count':newDF.iloc[i[0]].rating_count,
                'product_description':newDF.iloc[i[0]].description,
                'product_fetch_date':newDF.iloc[i[0]].date,
                'product_store':newDF.iloc[i[0]].product_store,
                'product_weighted_rating': newDF.iloc[i[0]].rating_weighted,
            }
            recommended_products.append(product_info)

    return recommended_products

In [39]:
recommendProducts("iPhone 15")

[{'product_id': 2205,
  'product_name': 'Apple iPhone 15 512GB',
  'product_link': 'https://www.shophive.com/apple-iphone-15-512gb/',
  'product_image': 'https://www.shophive.com/media/catalog/product/cache/3875881abdd255ac261538b8462285e9/i/p/iphone_15_1_1_1.jpg',
  'product_price': 0.0,
  'product_category': 'Apple, iPhone',
  'product_ratings': 4.631923764145324,
  'product_rating_count': 17.65276950565813,
  'product_description': '{\'Brand\': \'Apple\', \'Operating System\': \'Apple iOS\', \'Screen Size\': \'6.1"\', \'Internal Memory\': \'512GB\', \'RAM\': \'6GB\', \'Main Camera Pixel\': \'48\', \'Dual Sim\': \'No\', \'Water Resistant\': \'Yes\', \'Expandable Memory\': \'No\', \'Warranty\': \'7 Day Replacement\', \'Shipping\': \'On Order\', \'Color\': \'Black\'}',
  'product_fetch_date': '2023-12-03',
  'product_store': 'Shophive',
  'product_weighted_rating': 81.76628257623781},
 {'product_id': 2204,
  'product_name': 'Apple iPhone 15 Pro 512GB',
  'product_link': 'https://www.sh

# PRICE COMPARISON

In [41]:
user_search = ""

In [42]:
matched_products = newDF[newDF['product_name'].str.contains(user_search, case=False)]

In [43]:

# Sort the matched products based on ratings and prices
# matched_products_sorted = matched_products.sort_values(by=['rating_weighted', 'product_price'], ascending=[False, True])

# Sort on Price Alone
matched_products_sorted = matched_products.sort_values(by='product_price', ascending=True)


In [44]:
matched_products_sorted

,product_name,product_price,product_image,product_link,product_store,product_category,product_ratings,rating_count,description,date,rating_weighted,product_id,tags
2398,Apple Watch SE 44mm Starlight Case With Starli...,0.0,https://www.shophive.com/media/catalog/product...,https://www.shophive.com/apple-watch-se-44mm-s...,Shophive,"SmartWatches, Apple",4.631924,17.65277,"{'Brand': 'Apple', 'Shipping': 'Same Day From ...",2023-12-03,81.766283,2399,apple watch se 44mm starlight case with starli...
1706,Apple MacBook Pro MRW63 M3 Pro 16 Inch 512GB S...,0.0,https://www.shophive.com/media/catalog/product...,https://www.shophive.com/apple-macbook-pro-mrw...,Shophive,"Laptops, Mac, Apple, Moshi",4.631924,17.65277,"{'Brand': 'Apple', 'Series': 'Macbook Pro', 'S...",2023-12-03,81.766283,1707,apple macbook pro mrw63 m3 pro 16 inch 512gb s...
1707,"Apple Macbook Pro MRX83 M3 Max 14.2"" 1TB SSD S...",0.0,https://www.shophive.com/media/catalog/product...,https://www.shophive.com/apple-macbook-pro-mrx...,Shophive,"Laptops, Mac, Apple, Moshi",4.631924,17.65277,"{'Brand': 'Apple', 'Series': 'Macbook Pro', 'S...",2023-12-03,81.766283,1708,apple macbook pro mrx83 m3 max 14 2 1tb ssd si...
1708,Apple MacBook Pro MUW73 M3 Max 16 Inch 1TB SSD...,0.0,https://www.shophive.com/media/catalog/product...,https://www.shophive.com/apple-macbook-pro-muw...,Shophive,"Laptops, Mac, Apple, Moshi",4.631924,17.65277,"{'Brand': 'Apple', 'Series': 'Macbook Pro', 'S...",2023-12-03,81.766283,1709,apple macbook pro muw73 m3 max 16 inch 1tb ssd...
1709,"Apple Macbook Pro MR7K3 M3 14.2"" 1TB SSD Silve...",0.0,https://www.shophive.com/media/catalog/product...,https://www.shophive.com/apple-macbook-pro-mr7...,Shophive,"Laptops, Mac, Apple, Moshi",4.631924,17.65277,"{'Brand': 'Apple', 'Series': 'Macbook Pro', 'S...",2023-12-03,81.766283,1710,apple macbook pro mr7k3 m3 14 2 1tb ssd silver...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1728,"Apple Macbook Pro 2023 M2 MAX 16"" 96GB 2TB SSD...",1764999.0,https://www.shophive.com/media/catalog/product...,https://www.shophive.com/apple-macbook-pro-202...,Shophive,"Laptops, Mac, Apple, Moshi",4.631924,17.65277,"{'Brand': 'Apple', 'Series': 'Macbook Pro', 'S...",2023-12-03,81.766283,1729,apple macbook pro 2023 m2 max 16 96gb 2tb ssd ...
2049,"Apple Macbook Pro 2023 M2 MAX 16"" 96GB 2TB SSD...",1764999.0,https://www.shophive.com/media/catalog/product...,https://www.shophive.com/apple-macbook-pro-202...,Shophive,"Apple, Moshi, Targus, Ugreen, Wiwu, Macbook Ai...",4.631924,17.65277,"{'Brand': 'Apple', 'Series': 'Macbook Pro', 'S...",2023-12-03,81.766283,2050,apple macbook pro 2023 m2 max 16 96gb 2tb ssd ...
2047,"Apple MacBook Pro 16.2"" M2 Max 64GB RAM 4TB SS...",1869999.0,https://www.shophive.com/media/catalog/product...,https://www.shophive.com/apple-macbook-pro-16-...,Shophive,"Apple, Moshi, Targus, Ugreen, Wiwu, Macbook Ai...",4.631924,17.65277,"{'Brand': 'Apple', 'Series': 'Macbook Pro', 'S...",2023-12-03,81.766283,2048,apple macbook pro 16 2 m2 max 64gb ram 4tb ssd...
1726,"Apple MacBook Pro 16.2"" M2 Max 64GB RAM 4TB SS...",1869999.0,https://www.shophive.com/media/catalog/product...,https://www.shophive.com/apple-macbook-pro-16-...,Shophive,"Laptops, Mac, Apple, Moshi",4.631924,17.65277,"{'Brand': 'Apple', 'Series': 'Macbook Pro', 'S...",2023-12-03,81.766283,1727,apple macbook pro 16 2 m2 max 64gb ram 4tb ssd...


# Collaborative Based Recommendations

In [45]:
products = newDF

### Generating User Data 

In [298]:
from faker import Faker
import random

# Initialize the Faker generator
fake = Faker()

# Extract product IDs from the DataFrame
product_ids = products['product_id'].tolist()

# Function to generate fake rating data with some users not rating all items
def generate_fake_ratings(num_users, product_ids):
    ratings = []
    for user_id in range(1, num_users + 1):
        # Determine how many products the user will rate (randomly)
        num_rated_products = random.randint(1, len(product_ids))
        # Randomly choose products to be rated
        rated_products = random.sample(product_ids, num_rated_products)
        for product_id in rated_products:
            rating = random.randint(1, 5)
            ratings.append({'user_id': user_id, 'product_id': product_id, 'rating': rating})
    return ratings

# Generate fake rating data for 1000 users and product IDs
num_users = 2000
fake_ratings = generate_fake_ratings(num_users, product_ids)
# Convert the list of dictionaries to a DataFrame
ratings = pd.DataFrame(fake_ratings)

In [355]:
ratings

,user_id,product_id,rating
0,1,2272,3
1,1,1612,4
2,1,3661,3
3,1,60,3
4,1,2750,5
...,...,...,...
3713559,2000,1802,5
3713560,2000,2297,3
3713561,2000,3090,1
3713562,2000,2946,3


## Collaborative Filtering

In [300]:
products.head()

,product_name,product_price,product_image,product_link,product_store,product_category,product_ratings,rating_count,description,date,rating_weighted,product_id,tags
0,Mibro A2 Bluetooth Calling Smart Watch,8299.0,https://images.priceoye.pk/mibro-a2-bluetooth-...,https://priceoye.pk/smart-watches/mibro/mibro-...,PriceOye,"Smart-Watches, Mibro",5.0,8.0,"{'Display': '1.39 inches', 'Screen Type': 'HD ...",2023-12-03,40.0,1,mibro a2 bluetooth calling smart watch display...
1,Mibro Watch Lite2,12999.0,https://images.priceoye.pk/mibro-watch-lite2-p...,https://priceoye.pk/smart-watches/mibro/mibro-...,PriceOye,"Smart-Watches, Mibro",5.0,79.0,"{'Display': '1.3 Inches', 'Screen Type': 'Amol...",2023-12-03,395.0,2,mibro watch lite2 display 1 3 inch screen type...
2,Mibro GS Pro Calling Smart Watch,15999.0,https://images.priceoye.pk/mibro-gs-pro-callin...,https://priceoye.pk/smart-watches/mibro/mibro-...,PriceOye,"Smart-Watches, Mibro",5.0,5.0,"{'Display': '1.43', 'Screen Type': 'AMOLED', '...",2023-12-03,25.0,3,mibro g pro calling smart watch display 1 43 s...
3,Mibro T1 Smart Watch,13999.0,https://images.priceoye.pk/mibro-t1-pakistan-p...,https://priceoye.pk/smart-watches/mibro/mibro-t1,PriceOye,"Smart-Watches, Mibro",5.0,79.0,"{'Display': '1.6 Inch', 'Screen Type': 'AMOLED...",2023-12-03,395.0,4,mibro t1 smart watch display 1 6 inch screen t...
4,Mibro Smart Watch T2,13999.0,https://images.priceoye.pk/mibro-smart-watch-t...,https://priceoye.pk/smart-watches/mibro/mibro-...,PriceOye,"Smart-Watches, Mibro",5.0,1.0,"{'Display': '1.75 Inches', 'Screen Type': 'AMO...",2023-12-03,5.0,5,mibro smart watch t2 display 1 75 inch screen ...


In [301]:
# Merge the DataFrames on the product_id column
merged_df = pd.merge(ratings, products, on='product_id', how='inner')

In [302]:
merged_df.head()

,user_id,product_id,rating,product_name,product_price,product_image,product_link,product_store,product_category,product_ratings,rating_count,description,date,rating_weighted,tags
0,1,2272,3,ESR Apple iPhone 13 Pro Max MagSafe Cloud Supe...,4299.0,https://www.shophive.com/media/catalog/product...,https://www.shophive.com/esr-apple-iphone-13-p...,Shophive,"Apple, iPhone",4.631924,17.65277,"{'Brand': 'ESR', 'Warranty': 'Check at Deliver...",2023-12-03,81.766283,esr apple iphone 13 pro max magsafe cloud supe...
1,3,2272,1,ESR Apple iPhone 13 Pro Max MagSafe Cloud Supe...,4299.0,https://www.shophive.com/media/catalog/product...,https://www.shophive.com/esr-apple-iphone-13-p...,Shophive,"Apple, iPhone",4.631924,17.65277,"{'Brand': 'ESR', 'Warranty': 'Check at Deliver...",2023-12-03,81.766283,esr apple iphone 13 pro max magsafe cloud supe...
2,4,2272,4,ESR Apple iPhone 13 Pro Max MagSafe Cloud Supe...,4299.0,https://www.shophive.com/media/catalog/product...,https://www.shophive.com/esr-apple-iphone-13-p...,Shophive,"Apple, iPhone",4.631924,17.65277,"{'Brand': 'ESR', 'Warranty': 'Check at Deliver...",2023-12-03,81.766283,esr apple iphone 13 pro max magsafe cloud supe...
3,5,2272,1,ESR Apple iPhone 13 Pro Max MagSafe Cloud Supe...,4299.0,https://www.shophive.com/media/catalog/product...,https://www.shophive.com/esr-apple-iphone-13-p...,Shophive,"Apple, iPhone",4.631924,17.65277,"{'Brand': 'ESR', 'Warranty': 'Check at Deliver...",2023-12-03,81.766283,esr apple iphone 13 pro max magsafe cloud supe...
4,7,2272,5,ESR Apple iPhone 13 Pro Max MagSafe Cloud Supe...,4299.0,https://www.shophive.com/media/catalog/product...,https://www.shophive.com/esr-apple-iphone-13-p...,Shophive,"Apple, iPhone",4.631924,17.65277,"{'Brand': 'ESR', 'Warranty': 'Check at Deliver...",2023-12-03,81.766283,esr apple iphone 13 pro max magsafe cloud supe...


In [308]:
merged_df.groupby('user_id').count()['rating'] > 3000

user_id
1       False
2       False
3       False
4        True
5       False
        ...  
1996     True
1997     True
1998     True
1999     True
2000    False
Name: rating, Length: 2000, dtype: bool

In [309]:
x = merged_df.groupby('user_id').count()['rating'] > 3000
filtered_users = x[x].index

In [310]:
filtered_rating = merged_df[merged_df['user_id'].isin(filtered_users)]

In [342]:
y = filtered_rating.groupby('product_name').count()['rating'] >= 200
famous_prod = y[y].index

In [343]:
famous_prod

Index(['A4Tech FH200i Conference Single Pin Over-Ear Headphone',
       'A4Tech HS-50 - Wired - Over the Ear - Headphones, Stereo Head Set With Mic/ Earphone Hook /Volume & Mic Control In Line',
       'A4Tech HS-5P Internet Headset', 'A4Tech HS-7P Headset',
       'A4Tech HS-8i Stereo Headset',
       'A4Tech HU-35 Comfort Fit Stereo USB Headset',
       'A4Tech HU-7P Comfort Fit Stereo Headset',
       'A4Tech HU-8 USB Stereo Headset',
       'A4tech 2Drumtek B20 True Wireless Earphone - Ash Grey',
       'A4tech 2Drumtek B20 True Wireless Earphone - Grayish White',
       ...
       'vivo iQOO Neo', 'vivo iQOO Neo 6', 'vivo iQOO Neo5 S', 'vivo iQOO Pro',
       'vivo iQOO U1', 'vivo iQOO U3', 'vivo iQOO U5x', 'vivo iQOO Z3',
       'vivo iQOO Z6', 'vivo v19 pro'],
      dtype='object', name='product_name', length=3542)

In [344]:
final_ratings = filtered_rating[filtered_rating['product_name'].isin(famous_prod)]

In [345]:
final_ratings

,user_id,product_id,rating,product_name,product_price,product_image,product_link,product_store,product_category,product_ratings,rating_count,description,date,rating_weighted,tags
2,4,2272,4,ESR Apple iPhone 13 Pro Max MagSafe Cloud Supe...,4299.0,https://www.shophive.com/media/catalog/product...,https://www.shophive.com/esr-apple-iphone-13-p...,Shophive,"Apple, iPhone",4.631924,17.65277,"{'Brand': 'ESR', 'Warranty': 'Check at Deliver...",2023-12-03,81.766283,esr apple iphone 13 pro max magsafe cloud supe...
7,11,2272,5,ESR Apple iPhone 13 Pro Max MagSafe Cloud Supe...,4299.0,https://www.shophive.com/media/catalog/product...,https://www.shophive.com/esr-apple-iphone-13-p...,Shophive,"Apple, iPhone",4.631924,17.65277,"{'Brand': 'ESR', 'Warranty': 'Check at Deliver...",2023-12-03,81.766283,esr apple iphone 13 pro max magsafe cloud supe...
11,22,2272,1,ESR Apple iPhone 13 Pro Max MagSafe Cloud Supe...,4299.0,https://www.shophive.com/media/catalog/product...,https://www.shophive.com/esr-apple-iphone-13-p...,Shophive,"Apple, iPhone",4.631924,17.65277,"{'Brand': 'ESR', 'Warranty': 'Check at Deliver...",2023-12-03,81.766283,esr apple iphone 13 pro max magsafe cloud supe...
13,24,2272,1,ESR Apple iPhone 13 Pro Max MagSafe Cloud Supe...,4299.0,https://www.shophive.com/media/catalog/product...,https://www.shophive.com/esr-apple-iphone-13-p...,Shophive,"Apple, iPhone",4.631924,17.65277,"{'Brand': 'ESR', 'Warranty': 'Check at Deliver...",2023-12-03,81.766283,esr apple iphone 13 pro max magsafe cloud supe...
14,25,2272,4,ESR Apple iPhone 13 Pro Max MagSafe Cloud Supe...,4299.0,https://www.shophive.com/media/catalog/product...,https://www.shophive.com/esr-apple-iphone-13-p...,Shophive,"Apple, iPhone",4.631924,17.65277,"{'Brand': 'ESR', 'Warranty': 'Check at Deliver...",2023-12-03,81.766283,esr apple iphone 13 pro max magsafe cloud supe...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3713555,1984,2441,3,Apple Watch Series 8 41mm Graphite Stainless S...,0.0,https://www.shophive.com/media/catalog/product...,https://www.shophive.com/apple-watch-series-8-...,Shophive,"SmartWatches, Apple",4.631924,17.65277,"{'Brand': 'Apple', 'Shipping': 'On Order'}",2023-12-03,81.766283,apple watch series 8 41mm graphite stainless s...
3713558,1991,2441,3,Apple Watch Series 8 41mm Graphite Stainless S...,0.0,https://www.shophive.com/media/catalog/product...,https://www.shophive.com/apple-watch-series-8-...,Shophive,"SmartWatches, Apple",4.631924,17.65277,"{'Brand': 'Apple', 'Shipping': 'On Order'}",2023-12-03,81.766283,apple watch series 8 41mm graphite stainless s...
3713560,1996,2441,5,Apple Watch Series 8 41mm Graphite Stainless S...,0.0,https://www.shophive.com/media/catalog/product...,https://www.shophive.com/apple-watch-series-8-...,Shophive,"SmartWatches, Apple",4.631924,17.65277,"{'Brand': 'Apple', 'Shipping': 'On Order'}",2023-12-03,81.766283,apple watch series 8 41mm graphite stainless s...
3713561,1998,2441,2,Apple Watch Series 8 41mm Graphite Stainless S...,0.0,https://www.shophive.com/media/catalog/product...,https://www.shophive.com/apple-watch-series-8-...,Shophive,"SmartWatches, Apple",4.631924,17.65277,"{'Brand': 'Apple', 'Shipping': 'On Order'}",2023-12-03,81.766283,apple watch series 8 41mm graphite stainless s...


In [346]:
pt = final_ratings.pivot_table(index='product_name',columns='user_id',values='rating')

# this pivot table

In [347]:
pt.fillna(0,inplace=True)

In [348]:
pt

user_id,4,11,22,24,25,26,31,34,35,41,...,1945,1952,1953,1983,1984,1991,1996,1997,1998,1999
product_name,,,,,,,,,,,,,,,,,,,,,
A4Tech FH200i Conference Single Pin Over-Ear Headphone,1.0,1.0,2.0,2.0,4.0,4.0,0.0,4.0,3.0,2.0,...,4.0,5.0,4.0,4.0,2.0,3.0,4.0,4.0,3.0,5.0
"A4Tech HS-50 - Wired - Over the Ear - Headphones, Stereo Head Set With Mic/ Earphone Hook /Volume & Mic Control In Line",1.0,4.0,2.0,2.0,1.0,2.0,2.0,2.0,4.0,4.0,...,1.0,3.0,5.0,4.0,1.0,4.0,5.0,3.0,1.0,3.0
A4Tech HS-5P Internet Headset,0.0,1.0,2.0,3.0,1.0,1.0,5.0,4.0,2.0,4.0,...,0.0,2.0,0.0,4.0,1.0,1.0,2.0,0.0,1.0,4.0
A4Tech HS-7P Headset,4.0,4.0,5.0,5.0,1.0,1.0,4.0,5.0,2.0,3.0,...,5.0,1.0,5.0,1.0,2.0,5.0,5.0,5.0,5.0,2.0
A4Tech HS-8i Stereo Headset,5.0,5.0,3.0,3.0,3.0,5.0,3.0,4.0,4.0,1.0,...,4.0,3.0,4.0,2.0,5.0,5.0,1.0,4.0,2.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
vivo iQOO U3,4.0,1.0,1.0,5.0,4.0,1.0,4.0,4.0,4.0,0.0,...,4.0,1.0,3.0,4.0,3.0,3.0,2.0,4.0,1.0,4.0
vivo iQOO U5x,3.0,5.0,3.0,1.0,1.0,2.0,1.0,3.0,2.0,2.0,...,4.0,5.0,2.0,1.0,1.0,4.0,1.0,4.0,5.0,1.0
vivo iQOO Z3,3.0,4.0,1.0,1.0,1.0,4.0,5.0,1.0,1.0,4.0,...,1.0,2.0,5.0,3.0,3.0,5.0,1.0,2.0,2.0,4.0


In [349]:
similarity_score = cosine_similarity(pt)

In [357]:
def recommend(movie_name, productsdf):
    recommended_products = []
    
    # Assume that 'movie_name' corresponds to a product_name in 'productsdf'
    matching_products = productsdf[productsdf['product_name'].str.contains(movie_name, case=False)]

    if not matching_products.empty:
        product_id = matching_products.iloc[0]['product_id']
        
        # Get the index of the product_id in 'productsdf'
        index = productsdf.index[productsdf['product_id'] == product_id][0]
        
        # Assuming similarity_score is calculated using 'productsdf' and is available
        similar_items = sorted(list(enumerate(similarity_score[index])), key=lambda x: x[1], reverse=True)[1:10]
    
        for i in similar_items:
            product_info = {
                'product_id': productsdf.iloc[i[0]]['product_id'],
                'product_name': productsdf.iloc[i[0]]['product_name'],
                'product_link': productsdf.iloc[i[0]]['product_link'],
                'product_image': productsdf.iloc[i[0]]['product_image'],
                'product_price': productsdf.iloc[i[0]]['product_price'],
                'product_category': productsdf.iloc[i[0]]['product_category'],
                'product_ratings': productsdf.iloc[i[0]]['product_ratings'],
                'product_rating_count': productsdf.iloc[i[0]]['rating_count'],
                'product_description': productsdf.iloc[i[0]]['description'],
                'product_fetch_date': productsdf.iloc[i[0]]['date'],
                'product_store': productsdf.iloc[i[0]]['product_store'],
                'product_weighted_rating': productsdf.iloc[i[0]]['rating_weighted'],
            }
            recommended_products.append(product_info)

    return recommended_products


In [381]:
recommend('Samsung Galaxy', newDF)

[{'product_id': 166,
  'product_name': 'Apple MacBook Pro 13.3 Inches M1 Chip 8GB RAM - 256GB SSD (MYD82)',
  'product_link': 'https://priceoye.pk/laptops/apple/apple-macbook-pro-13-3-inches-m1-chip-8gb-ram-256gb-ssd-myd82',
  'product_image': 'https://images.priceoye.pk/apple-macbook-pro-13-inches-m1-chip-8gb-ram-256gb-ssd-myd82-pakistan-priceoye-ttk0m-270x270.webp',
  'product_price': 240000.0,
  'product_category': 'Laptops, Apple, Moshi, Targus, Ugreen, Wiwu, Macbook Air, Macbook Pro',
  'product_ratings': 5.0,
  'product_rating_count': 1.0,
  'product_description': "{'Display': '13.3', 'Generation': 'M1', 'Processor': 'Apple M1 Chip', 'Internal Memory': '256GB SSD'}",
  'product_fetch_date': '2023-12-03',
  'product_store': 'PriceOye',
  'product_weighted_rating': 5.0},
 {'product_id': 1023,
  'product_name': 'Xiaomi Redmi Note 4',
  'product_link': 'https://priceoye.pk/mobiles/xiaomi/xiaomi-redmi-note-4',
  'product_image': 'https://images.priceoye.pk/xiaomi-redmi-note-4-pakistan

# Hybrid Recommendations

In [370]:
def get_hybrid_recommendations(product_name, products):
    content_based = recommendProducts(product_name)
    collaborative_based = recommend(product_name, products)
    hybrid_recommendations = {'Coontent Based Recommendations' : content_based,
                             'Collaborative Based Recommendations' : collaborative_based} 
    return hybrid_recommendations

In [380]:
get_hybrid_recommendations("Samsung Galaxy", newDF)

{'Coontent Based Recommendations': [{'product_id': 288,
   'product_name': 'Samsung Galaxy M23',
   'product_link': 'https://priceoye.pk/mobiles/samsung/samsung-galaxy-m23',
   'product_image': 'https://images.priceoye.pk/samsung-galaxy-m23-pakistan-priceoye-lgspd-270x270.webp',
   'product_price': 44999.0,
   'product_category': 'MobilePhones, Samsung',
   'product_ratings': 5.0,
   'product_rating_count': 1.0,
   'product_description': "{'Display': '6.6 Inches', 'RAM': '4 GB', 'Battery': '5000 mAh', 'Back Camera': '50 MP + 8 MP + 2 MP'}",
   'product_fetch_date': '2023-12-03',
   'product_store': 'PriceOye',
   'product_weighted_rating': 5.0},
  {'product_id': 192,
   'product_name': 'Samsung Galaxy A23',
   'product_link': 'https://priceoye.pk/mobiles/samsung/samsung-galaxy-a23',
   'product_image': 'https://images.priceoye.pk/samsung-galaxy-a23-pakistan-priceoye-ap7s8-270x270.webp',
   'product_price': 74199.0,
   'product_category': 'MobilePhones, Samsung',
   'product_ratings': 5

## Exporting

In [383]:
# Now i have the similarity matrix with the data i can extract these a and use them in my App and extension using the 
# Similarity Scores and Data


pickle.dump(newDF.to_dict(), open('products_dictionary.pkl', 'wb')) # Exporting the products as a dictionary
pickle.dump(similarity, open('similarity.pkl', 'wb')) # Exporting the content based similarity matrix
pickle.dump(similarity_score, open('collaborative_similarity.pkl', 'wb')) # Exporting the Collaborative Based Similarity Matrix